# Fix responses in the Continuous task

Something weird happened where the reaction times in the Continuous task are mostly negative.
This script corrects the responses by shifting participant answers to the previous trial.

It also discards the trials corresponding to the first two trials of each sweep.

For each participant (skipping over those that are listed in the `ignore_participants` list):

1. `fetch_initialization_data`: Initialization data is fetched for each participant.
2. `process_csv_files`: CSV files for each participant are processed and compiled into a single DataFrame.
3. `correct_data`: The data is corrected by addressing negative reaction times, removing catch trials, and discarding the first two trials of each sweep outside of the random condition. False alarm rates and detection ratios are computed and logged.
4. `recompute_audiograms`: external API call to recompute all four audiograms, using the initialization data and the corrected data.
5. `save_audiogram`: Each recomputed audiogram is saved in RAW_DATA/participant/Continuous with the suffix 'fixed'.

Finally, print stats on the effect of the corrections on the FAR and detection ratio.

In [1]:
from scripts.utils import *

from funcs.fix_continuous_responses import *

In [2]:
# Set path to raw data (csv files)
raw_data_path = get_path('raw_data')

# Choose participants to exclude
ignore_participants = exclude_participants()

overwrite_existing = True

In [0]:
# Call the main function
# process_participants(raw_data_path, ignore_participants, overwrite=True)

API_access = get_API_access()

os.chdir(raw_data_path)

API_calls = {}
API_answer = {}
init_data = {}

# Keep track of whether detection and false alarm ratios improve
are_FARs_looking_better = []
are_DRs_looking_better = []

for participant in os.listdir():
    if participant not in ignore_participants:
        API_calls[participant] = {}
        API_answer[participant] = {}
        os.chdir(f'./{participant}/Continuous')

        recomputed_files = list(filter(os.path.isfile, glob.glob("*_fixed.pkl")))

        os.chdir(raw_data_path)

        if len(recomputed_files) < 4 or overwrite_existing:
            print("\n--------------------", participant, "--------------------")

            init_data[participant] = fetch_initialization_data(participant, raw_data_path)

            participant_data = process_csv_files(participant, raw_data_path)

            participant_data, effect_on_FAR, effect_on_DR = correct_data(participant_data, participant)

            are_FARs_looking_better.append(effect_on_FAR)
            are_DRs_looking_better.append(effect_on_DR)

            for pred, pred_group in participant_data.groupby('Prediction'):

                API_answer[participant][pred] = recompute_audiogram(participant, pred, pred_group, init_data[participant], API_calls, *API_access)

                save_audiogram(raw_data_path, participant, pred, API_answer[participant][pred])

            print("-------------------------------------------------")

In [4]:

# Check if we've "improved" FARs and detection rations with the corrections made
print("")
print(f"Improved (decreased) FARs for {np.sum(np.array(are_FARs_looking_better) > 0)} participants")
print(f"Mean improvement: {np.round(np.mean(are_FARs_looking_better) * 100)} %")
print("")
print(f"Improved (increased) detection ratios for {np.sum(np.array(are_DRs_looking_better) > 0)} participants")
print(f"Mean improvement: {np.round(np.mean(are_DRs_looking_better) * 100)} %")
print("")



Improved (decreased) FARs for 27 participants
Mean improvement: 43.0 %

Improved (increased) detection ratios for 28 participants
Mean improvement: 5.0 %

